<a href="https://colab.research.google.com/github/kimheeseo/pytorch-RNN/blob/main/RNN_%EC%97%B0%EC%8A%B5_2021_12_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
#출처 : https://justkode.kr/deep-learning/pytorch-rnn
sentences = ["i like dog", "i love coffee", "i hate milk", "you like cat", "you love milk", "you hate coffee"]
dtype = torch.float

"""
Word Processing
"""
word_list = list(set(" ".join(sentences)))
print('word_list값',word_list)


word_list값 ['l', 'h', 'g', 'k', 'a', 'c', 'v', 'i', 'e', 't', 'f', 'o', 'u', ' ', 'y', 'd', 'm']


In [3]:
word_list = list(set(" ".join(sentences).split()))
print(word_list)
word_dict = {w: i for i, w in enumerate(word_list)}
print('word_dict값',word_dict)
number_dict = {i: w for i, w in enumerate(word_list)}
print('number_dict값',number_dict)
n_class = len(word_dict)
print('n_class값',n_class)

['cat', 'like', 'you', 'coffee', 'milk', 'i', 'hate', 'love', 'dog']
word_dict값 {'cat': 0, 'like': 1, 'you': 2, 'coffee': 3, 'milk': 4, 'i': 5, 'hate': 6, 'love': 7, 'dog': 8}
number_dict값 {0: 'cat', 1: 'like', 2: 'you', 3: 'coffee', 4: 'milk', 5: 'i', 6: 'hate', 7: 'love', 8: 'dog'}
n_class값 9


In [4]:
"""
TextRNN Parameter
"""
batch_size = len(sentences)
print('batch_size값',batch_size)
n_step = 2  # 학습 하려고 하는 문장의 길이 - 1
n_hidden = 5  # 은닉층 사이즈
print('sentences값',sentences)

inputtest_batch=[]
targettest_batch=[]
i=0
#다음 for문은 make_batch문의 동작을 보기위해서, 나눠서 표현방법을 본다.
for sen in sentences:
  print(i)
  i=i+1
  word=sen.split()
  print('word값',word)
  input=[word_dict[n] for n in word[:]]
  print('input값',input)
  target=word_dict[word[-1]]
  print('word[-1]값',word[-1])
  #print('np.eye(n_class)값',np.eye(n_class))
  #print('np.eye(n_class)[input]',np.eye(n_class)[input])
  inputtest_batch.append(np.eye(n_class)[input])  # One-Hot Encoding
  #print('inputtest_batch값',inputtest_batch)
  print('target값',target)
  targettest_batch.append(target)

print('inputtest_batch값',inputtest_batch)
print('targettest_batch값',targettest_batch)

#split에 의해 1번째 for문 동작 : 'I like dog'이 sen값이 된다. 그리고 "word=sen.split()"에 의해서
#word값은 ['I','like']가 되고, input값은 [2,0]이 된다. 우선 2개의 값이 나오는 이유는 [:-1]에 의해서 그런것이고,
#만일 [:]이면, ['I','like','dog']가 되고 input값은 [2,0,1]이 되고, 이때 숫자는 앞서 word_dict을 통해 설정한
#I는 2, like는 0, dog는 1을 나타내기 때문에, 이와 같은 결과값이 나온다.

batch_size값 6
sentences값 ['i like dog', 'i love coffee', 'i hate milk', 'you like cat', 'you love milk', 'you hate coffee']
0
word값 ['i', 'like', 'dog']
input값 [5, 1, 8]
word[-1]값 dog
target값 8
1
word값 ['i', 'love', 'coffee']
input값 [5, 7, 3]
word[-1]값 coffee
target값 3
2
word값 ['i', 'hate', 'milk']
input값 [5, 6, 4]
word[-1]값 milk
target값 4
3
word값 ['you', 'like', 'cat']
input값 [2, 1, 0]
word[-1]값 cat
target값 0
4
word값 ['you', 'love', 'milk']
input값 [2, 7, 4]
word[-1]값 milk
target값 4
5
word값 ['you', 'hate', 'coffee']
input값 [2, 6, 3]
word[-1]값 coffee
target값 3
inputtest_batch값 [array([[0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1.]]), array([[0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0.]]), array([[0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0.]]), array

In [5]:
def make_batch(sentences):
  input_batch = []
  target_batch = []

  for sen in sentences:
    word = sen.split()
    input = [word_dict[n] for n in word[:-1]]
    target = word_dict[word[-1]]

    input_batch.append(np.eye(n_class)[input])  # One-Hot Encoding
    target_batch.append(target)
  
  return input_batch, target_batch  

In [6]:
input_batch, target_batch = make_batch(sentences)
input_batch = torch.tensor(input_batch, dtype=torch.float32, requires_grad=True)
target_batch = torch.tensor(target_batch, dtype=torch.int64)
print('input_batch값',input_batch)
print('target_batch값',target_batch)

input_batch값 tensor([[[0., 0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 1., 0.]],

        [[0., 0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0.]],

        [[0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 1., 0.]],

        [[0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0.]]], requires_grad=True)
target_batch값 tensor([8, 3, 4, 0, 4, 3])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  


In [10]:
"""
TextRNN
"""
class TextRNN(nn.Module):
  def __init__(self):
    super(TextRNN, self).__init__()

    self.rnn = nn.RNN(input_size=n_class, hidden_size=n_hidden, dropout=0.3)
    self.W = nn.Parameter(torch.randn([n_hidden, n_class]).type(dtype))
    self.b = nn.Parameter(torch.randn([n_class]).type(dtype))
    self.Softmax = nn.Softmax(dim=1)

  def forward(self, hidden, X):
    X = X.transpose(0, 1)
    outputs, hidden = self.rnn(X, hidden)
    outputs = outputs[-1]  # 최종 예측 Hidden Layer
    print('outputs값',outputs)
    model = torch.mm(outputs, self.W) + self.b  # 최종 예측 최종 출력 층
    print('model값',model)
    return model
	
"""
Training
"""
model = TextRNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

for epoch in range(500):
  hidden = torch.zeros(1, batch_size, n_hidden, requires_grad=True)
  output = model(hidden, input_batch)
  loss = criterion(output, target_batch)

  if (epoch + 1) % 100 == 0:
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
  
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

input = [sen.split()[:2] for sen in sentences]
print('input값',input)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
       grad_fn=<SelectBackward0>)
model값 tensor([[ 0.8450, -0.1327, -1.8921,  0.3747,  0.8094, -2.1374, -5.7256, -0.8989,
          6.2768],
        [-5.2189, -1.5206, -2.6590,  6.3684,  2.0482, -5.3142, -6.7553, -4.8772,
          0.7610],
        [-0.8610, -1.2811, -2.4875,  1.5422,  5.2537, -6.3842, -6.7144, -3.4916,
          0.7615],
        [ 5.4806, -2.0751, -1.7359, -0.2982, -1.4840, -1.0725,  0.1235, -0.2224,
         -1.8148],
        [-3.4863, -2.0551,  0.9426,  1.9729,  5.9900, -5.7209, -2.5945, -5.0770,
         -4.7574],
        [ 0.0079, -2.7627, -3.1065,  4.1047,  0.1162, -4.5439, -2.7675, -3.0771,
         -5.8155]], grad_fn=<AddBackward0>)
outputs값 tensor([[-0.7308,  0.0103, -0.9987, -0.9151,  0.9867],
        [-0.8170,  0.9705, -0.9211,  0.6588,  0.9995],
        [ 0.8634,  0.9531, -0.9986, -0.1393,  0.9830],
        [ 0.9733, -0.4519, -0.8673, -0.0679, -0.5091],
        [ 0.9591,  0.9222,  0.6468,  0.9793,  0.7793],
        [ 0.99

In [8]:
hidden = torch.zeros(1, batch_size, n_hidden, requires_grad=True)
print('hidden값',hidden)
#data.max(1,keepdim=True)[1] 관련 자료 : https://blog.naver.com/PostView.nhn?blogId=na_young_1124&logNo=221455757665&from=search&redirect=Log&widgetTypeCall=true&directAccess=false
predict = model(hidden, input_batch).data.max(1, keepdim=True)[1]
predict2 = model(hidden, input_batch).data.max(1, keepdim=True)[0]
predict3 = model(hidden, input_batch).data.max(1, keepdim=True)

print('predict3값',predict3)
print('predict2값',predict2)
print('predict값',predict)
print([sen.split()[:2] for sen in sentences], '->', [number_dict[n.item()] for n in predict.squeeze()])

hidden값 tensor([[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]], requires_grad=True)
outputs값 tensor([[-0.0791,  0.9137,  0.8672, -0.7724, -0.5065],
        [ 0.5522,  0.8629, -0.9481,  0.8048, -0.6714],
        [-0.8946,  0.9990, -0.8853,  0.8496,  0.8713],
        [ 0.9989, -0.7762,  0.8860, -0.4805,  0.9986],
        [ 0.9997, -0.8565, -0.9393,  0.9239,  0.9976],
        [ 0.9835,  0.8427, -0.8664,  0.9422,  1.0000]],
       grad_fn=<SelectBackward0>)
outputs값 tensor([[-0.0791,  0.9137,  0.8672, -0.7724, -0.5065],
        [ 0.5522,  0.8629, -0.9481,  0.8048, -0.6714],
        [-0.8946,  0.9990, -0.8853,  0.8496,  0.8713],
        [ 0.9989, -0.7762,  0.8860, -0.4805,  0.9986],
        [ 0.9997, -0.8565, -0.9393,  0.9239,  0.9976],
        [ 0.9835,  0.8427, -0.8664,  0.9422,  1.0000]],
       grad_fn=<SelectBackward0>)
outputs값 tensor([[-0.0791,  0.9137, 

In [9]:
predict = model(hidden, input_batch).data.max(1, keepdim=True)[1]
print(number_dict[n.item()] for n in predict.squeeze())
print(predict.squeeze())

outputs값 tensor([[-0.0791,  0.9137,  0.8672, -0.7724, -0.5065],
        [ 0.5522,  0.8629, -0.9481,  0.8048, -0.6714],
        [-0.8946,  0.9990, -0.8853,  0.8496,  0.8713],
        [ 0.9989, -0.7762,  0.8860, -0.4805,  0.9986],
        [ 0.9997, -0.8565, -0.9393,  0.9239,  0.9976],
        [ 0.9835,  0.8427, -0.8664,  0.9422,  1.0000]],
       grad_fn=<SelectBackward0>)
<generator object <genexpr> at 0x7fec282146d0>
tensor([8, 3, 4, 0, 4, 3])
